! jt -r

In [89]:
from importlib import reload
reload(aux)
reload(con)
reload(dt)
reload(info)

<module 'src.info' from 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\src\\info.py'>

In [90]:
import pandas as pd
import src.daily_data as dt
import src.auxiliary_functions as aux
import src.condition as con
from datetime import time, datetime,date
import src.info as info
import src.in_out as io
import numpy as np
from src.density import Density, draw
import itertools

# Hazırlık Aşaması

**data** klasöründeki tüm dosya yolları alınıyor
```python
>>> path_list
Out []:
 ['C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\20180906_6A_U8.xlsx',
 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\20180906_6B_U8.xlsx',
 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\20180906_6C_U8.xlsx',
 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\20180907_6A_U8.xlsx',
 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\20180907_6B_U8.xlsx',
 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\20180907_6C_U8.xlsx']
```

In [2]:
path_list = io.get_path('C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\')

Ürün adları sırayla alınıyor
```py
>>> product_names
Out[]:
['6AU8', '6BU8', '6CU8', '6AU8', '6BU8', '6CU8']```

In [3]:
product_names = [info.get_productName(path) for path in path_list]

Tarihler sırayla alınıyor
```py
>>> date_list
Out[]:
[Timestamp('2018-09-06 00:00:00'),
 Timestamp('2018-09-06 00:00:00'),
 Timestamp('2018-09-06 00:00:00'),
 Timestamp('2018-09-07 00:00:00'),
 Timestamp('2018-09-07 00:00:00'),
 Timestamp('2018-09-07 00:00:00')]
```

In [4]:
date_list = [info.get_productDate(path) for path in path_list]

Hangi sütunlardaki verileri alacağımızı aşağıdaki listede belirityoruz

In [5]:
cols= ["Time","BID price","ASK price"]

Dosyalardan veriler dataframe olarak okunuyor ve listeye atılıyor.
Okunan_veriler'in yapısı: <br/>
```py
list(pd.DataFrame,pd.DataFrame.....)
```

In [6]:
okunan_veriler = [pd.read_excel(path,usecols=cols) for path in path_list]

Her bir DataFrame'deki sütunlar yeniden adlandırılıyor

In [7]:
columns=['time','bid_price','ask_price']

In [8]:
for df in okunan_veriler:
    df.columns = columns

In [9]:
time_stamp = okunan_veriler[0].time

# Mid Price Hesaplama ve Saatlik Bölme 

hesaplanan her bir mid_price **list_of_mid_price** adlı listeye atılıyor. Listenin yapısı aşağıdaki gibidir:
```py
list(pd.Series,pd.Series,pd.Series...)
```

In [10]:
list_of_mid_price = list(map(info.get_mid_price,okunan_veriler))

Bulunan mid price hiyerarşik indeksleme ile tek DataFrame'de birleştiriliyor.

In [11]:
arrays = [date_list,product_names]

In [12]:
df_mp = pd.DataFrame(data=list_of_mid_price, index = arrays)

Bu dataframe zaman sütunu eklenip saatlik olarak bölünüyor.

In [13]:
df_mp_t = df_mp.T

In [14]:
df_mp_t['time'] = time_stamp

```py
>>> len(list_of_hourly_mp)
Out[]: 23
>>> df.columns.levels
Out[]:FrozenList([[2018-09-06 00:00:00, 2018-09-07 00:00:00, 'time'], ['6AU8', '6BU8', '6CU8', '']])
```

In [15]:
list_of_hourly_mp = info.split_df(df_mp_t,23)

----

# Ürünler Eşleştiriliyor

```python
>>> non_duplicate_dates
Out[]:[Timestamp('2018-09-06 00:00:00'), Timestamp('2018-09-07 00:00:00')]
>>> non_duplicate_names
Out[]:['6AU8', '6BU8', '6CU8']
>>> paired_products
Out[]: [('6AU8', '6BU8'), ('6AU8', '6CU8'), ('6BU8', '6CU8')]
```

In [16]:
non_duplicate_dates = list(dict.fromkeys(date_list)) #tekrar eden tarihler kaldırıldı.

In [17]:
non_duplicate_names = list(dict.fromkeys(product_names)) #tekrar eden ürünler kaldırıldı.

In [18]:
paired_products = list(itertools.combinations(non_duplicate_names, 2)) #2'li kombinasyon kullanılarak ürünler eşleştiriliyor.

----

# Spread Hesaplanıyor

**list_of_hourly_mp** da hiyerarşi şu şekildedir: <br/>
* Önce saate erişiriz
* İkinci olarak tarihe erişiriz
* Son olarak ürüne erişiriz <br/>
**Buna göre spread hesaplamak için aşağıdaki yol izlenir:** <br/>
    1. Her bir saate eriş
    2. Her bir tarihe eriş
    3. Pairleri al
    4. *spread_hesapla(pair): return pd.Series*
    

In [19]:
values={'a_PNLTICK':10,
'a_TICKSIZE':0.0001,
'b_PNLTICK':6.25,
'b_TICKSIZE':0.0001}

In [20]:
def spread_al(df):
    spread_dict = {}
    for date in non_duplicate_dates:
        for pair in paired_products:
            mp_1 = df[pd.Timestamp(date)][pair[0]]
            mp_2 = df[pd.Timestamp(date)][pair[1]]
            spread_dict[(date,pair[0]+"_"+pair[1])] = aux.find_spread(mp_1,mp_2,values)
    new_df = pd.DataFrame(spread_dict)
    new_df['time'] = df.time
    return new_df

In [21]:
spread = [spread_al(df) for df in list_of_hourly_mp]

----

# Change Hesaplanıyor

In [22]:
pairs = [pair[0]+'_'+pair[1] for pair in paired_products]

In [23]:
def change_al(df):
    change_dict = {}
    for date in non_duplicate_dates:
        for pair in pairs:
            sub_df = df[date][pair]            
            change_dict[(date,pair)] = aux.find_change(sub_df)
    new_df = pd.DataFrame(change_dict)
    new_df['time'] = df.time
    return new_df

In [24]:
change_list = [change_al(df) for df in spread]

---

# Duration ve Amplitude Hesaplanıyor

Hesaplama ile birlikte birleştirme de yapılıyor.
Birleştirme koşulu: <br/>
1. Ürünler aynı olacak
2. Tarihler farklı olacak
3. Saat dilimi aynı olacak

In [25]:
def unnamed1(df):
    lengt = df.shape[1]
    df_list = []
    for i in range(lengt):
        df_list.append(aux.find_duramp(df.iloc[:,1]))
    return pd.concat(df_list).dropna()

In [26]:
def unnamed2(df):
    new_df = duzenle(df)
    product = {}
    for pair in pairs:
        product[pair] = unnamed1(new_df[pair])
    return product

In [27]:
def duzenle(df):
    new_df = df.set_index('time')
    new_df = new_df.T.reset_index(level=0,drop=True).T
    return new_df

In [28]:
duramp = [unnamed2(change) for change in change_list]

---

# Medyanı baz alarak verileri yeniden düzenliyoruz

**pozitif ve negatif medyan, duration ve amplitude için ayrı ayrı baz alınarak hesaplanmıştır**

In [88]:
for pair in pairs:
    for i in range(23)
    current = duramp[i][pair]
    con.single_scan(current)

['6AU8_6BU8', '6AU8_6CU8', '6BU8_6CU8']

23

In [92]:
duramp[22]['6AU8_6BU8']

,duration,amplitude
time,,
16:00:01,1.0,1.250
16:00:05,4.0,-9.375
16:00:07,2.0,8.125
16:00:26,19.0,-6.250
16:00:27,1.0,6.250
16:00:34,7.0,-6.250
16:00:42,8.0,10.000
16:00:43,1.0,-3.125
16:00:56,13.0,3.125
